In [21]:
import pandas as pd
import numpy as np
data = pd.read_csv("raw_predictors_until2013.csv")
data

,yyyymm,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx
0,192701,13.21,0.6967,1.2290,0.443706,0.0323,0.0466,0.0561,0.0351,0.050834,0.002692,-0.011299,0.0075,0.0056,0.000470,NaN,-0.002910,-0.005579
1,192702,13.84,0.7033,1.2180,0.428501,0.0329,0.0467,0.0559,0.0347,0.051682,0.002742,-0.005714,0.0088,0.0069,0.000287,NaN,0.045522,0.040566
2,192703,13.93,0.7100,1.2080,0.469765,0.0320,0.0462,0.0554,0.0331,0.046370,0.002667,-0.005747,0.0253,0.0083,0.000924,NaN,0.007324,0.002610
3,192704,14.17,0.7167,1.1970,0.456754,0.0339,0.0458,0.0548,0.0333,0.050518,0.002825,0.000000,-0.0005,0.0055,0.000603,NaN,0.013021,0.010907
4,192705,14.91,0.7233,1.1860,0.434783,0.0333,0.0457,0.0550,0.0327,0.055279,0.002775,0.005780,0.0109,-0.0011,0.000392,NaN,0.062353,0.057096
5,192706,14.77,0.7300,1.1750,0.452385,0.0307,0.0458,0.0555,0.0334,0.058826,0.002558,0.011494,-0.0069,0.0043,0.000825,NaN,-0.020370,-0.025705
6,192707,15.73,0.7367,1.1640,0.414553,0.0296,0.0460,0.0555,0.0333,0.059754,0.002467,-0.017045,0.0050,0.0003,0.000426,NaN,0.084450,0.081687
7,192708,16.43,0.7433,1.1530,0.396227,0.0270,0.0456,0.0548,0.0329,0.054526,0.002250,-0.005780,0.0076,0.0083,0.001276,NaN,0.033320,0.028624
8,192709,17.14,0.7500,1.1430,0.380586,0.0268,0.0454,0.0542,0.0330,0.094617,0.002233,0.005814,0.0018,0.0149,0.001123,NaN,0.053016,0.049095
9,192710,16.23,0.7567,1.1320,0.413801,0.0308,0.0451,0.0538,0.0325,0.094370,0.002567,0.005780,0.0099,0.0055,0.001559,NaN,-0.046804,-0.048960


1092

In [6]:
sLength = len(data['yyyymm'])
# add the bullMarket column with random ints (to be replaced in next cell)
data['bull'] = pd.Series(np.random.randn(sLength), index=data.index)
# Create bullMarkert for ret_sp500+1
bull = []
for data in data['CRSP_SPvw']:
    if data > 0:
        bull.append(1)
    else:
        bull.append(0)
data['bull'] = bull

data.shape

TypeError: 'float' object is not subscriptable

In [3]:

# Rename columns
sp500.columns = ['Date', 'sp500', 'sp500+1','ret_sp500', 'ret_sp500+1']
sp500 = sp500.dropna()
sLength = len(sp500['Date'])
# add the bullMarket column with random ints (to be replaced in next cell)
sp500['bullMarket'] = pd.Series(np.random.randn(sLength), index=sp500.index)

# Create bullMarkert for ret_sp500+1
bull = []
for data in sp500['ret_sp500+1']:
    if data > 0:
        bull.append(1)
    else:
        bull.append(0)
sp500['bullMarket'] = bull
# print(sp500)

sp500.shape
# print(sp500)

(14275, 6)

In [27]:
tdata = pd.read_csv("Total_Data_no_dots.csv",low_memory=False)

result = pd.merge(sp500, tdata, on='Date', how='left')

result  = result.dropna()

# assume we want to predict the raw index of sp500 at T+1, given all the info at T

y_reg = result['ret_sp500+1']

y_cal = result['bullMarket']
 
# list(result)

result = result.drop(columns=['VOX Close', 'DGS1', 'DGS10', 'T10Y2Y', 'DBAA', 'USD1MTD156N', 'DCOILBRENTEU', 'DTWEXB', 'DEXUSEU', 'T10YIE', 'T5YIFR', 'BAMLH0A0HYM2EY'])

x = result.drop(columns=['Date','sp500_x','sp500+1','ret_sp500+1','Unnamed: 0'])

# print(result.dtypes)

In [29]:
# Linear Regression Model

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_validation import KFold

kf = KFold(x.shape[0], n_folds=2, shuffle=True)

for train_index, test_index in kf:
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y_reg.iloc[train_index], y_reg.iloc[test_index]
    y_train_cal, y_test_cal = y_cal.iloc[train_index], y_cal.iloc[test_index]

    regr = linear_model.LinearRegression()
    regr.fit(x_train, y_train)

    y_pred = regr.predict(x_test)
    print("MSE: ", mean_squared_error(y_test, y_pred))
    
print( "the main take way: the mean squared error is always within 1% ")


MSE:  0.8108592714839336
MSE:  0.8220161493992437
the main take way: the mean squared error is always within 1% 


In [31]:
# clssification 

# decision tree CART algorithm

from sklearn import tree
from sklearn.metrics import confusion_matrix
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train_cal)
predicted = clf.predict(x_test)
# print (predicted)
c_matrix = confusion_matrix(predicted,y_test_cal)
precision = c_matrix[1,1]/(c_matrix[1,1]+c_matrix[0,1])
recall= c_matrix[1,1]/(c_matrix[1,1]+c_matrix[1,0])

print(c_matrix)  

print("precision: ", precision)  
print("recall: ", recall)  

print( "the main take way: 'perfect' prediction, overfitting problem")


[[2132    0]
 [   0 2242]]
precision:  1.0
recall:  1.0
the main take way: 'perfect' prediction, overfitting problem


In [26]:
# naive Bayes 
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(x_train, y_train_cal).predict(x_test)

predicted= gnb.predict(x_test)

c_matrix = confusion_matrix(predicted,y_test_cal)
precision = c_matrix[1,1]/(c_matrix[1,1]+c_matrix[0,1])
recall= c_matrix[1,1]/(c_matrix[1,1]+c_matrix[1,0])

print(c_matrix)  

print("precision: ", precision)  
print("recall: ", recall)  

print( "the main take way: High Precision, recall is slightly better than a coin flip")

[[ 120  143]
 [2012 2099]]
precision:  0.9362176628010704
recall:  0.5105813670639747


In [34]:
# Support Vector Classification

from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(x_train, y_train_cal) 

predicted= clf.predict(x_test)

c_matrix = confusion_matrix(predicted,y_test_cal)
precision = c_matrix[1,1]/(c_matrix[1,1]+c_matrix[0,1])
recall= c_matrix[1,1]/(c_matrix[1,1]+c_matrix[1,0])

print(c_matrix)  

print("precision: ", precision)  
print("recall: ", recall)  

print( "the main take way: High Precision, recall is way more better than naive Bayes ")

[[1063    0]
 [1069 2242]]
precision:  1.0
recall:  0.6771368166717004
the main take way: High Precision, recall is way more better than naive Bayes 
